In [17]:
import os
import pickle
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np


input_dir = os.getcwd()
categories = ['genuino', 'no_genuino']

data = []
labels = []

# Cargamos imagenes divididas en dos carpetas, una de cada
# categoria. Se redimensionan y serializan.
for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path)
        img = resize(img, (30, 30))
        data.append(img.flatten())   # Flatted genera array unidimensional
        labels.append(category_idx)  # necesario para este modelo.
data = np.asarray(data)

# Preparamos el set de entrenamiento y prueba.
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, shuffle=True, stratify=labels)

# Entrenamos el modelo.
classifier = SVC()

# Prueba de hyperparametros para el modelo utilizado.
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]
grid_search = GridSearchCV(classifier, parameters)
grid_search.fit(x_train, y_train)
best_estimator = grid_search.best_estimator_

# Predecimos y comparamos. Tomamos metricas de eficiencia.
y_prediction = best_estimator.predict(x_test)
score = accuracy_score(y_prediction, y_test)
msg = '{}% de las muestras fueron correctamente clasificadas.'
print(msg.format(str(score * 100)))

# Almaceno modelo para futura utilizacion
pickle.dump(best_estimator, open('./model.p', 'wb'))

86.36363636363636% de las muestras fueron correctamente clasificadas.
